In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# Load data
df = pd.read_csv("../data/raw/balance_sample.csv")
df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y")
df = df.sort_values(by=['account', 'date'], ignore_index=True)
df.set_index('date', inplace=True)
df

,account,balance,currency
date,,,
2023-04-14,account1,5609.31,EUR
2023-07-28,account1,5581.59,EUR
2023-09-30,account1,6793.87,EUR
2023-11-13,account1,4104.02,EUR
2023-11-23,account1,123.69,EUR
...,...,...,...
2023-12-14,stocks2,2812.54,EUR
2024-02-11,stocks2,8588.81,EUR
2024-06-16,stocks2,4372.09,EUR


In [3]:
accounts = pd.read_csv("../data/raw/accounts_sample.csv")
accounts

,account,bank,type
0,account1,BNP Paribas,checking
1,account2,Keytrade Bank,checking
2,account3,BNP Paribas,savings
3,account4,Keytrade Bank,savings
4,stocks1,Keytrade Bank,stocks
5,stocks2,Degiro,stocks
6,ctrypto5,NaN,crypto


In [5]:
# Resample on daily frequency
df_resampled = df.groupby('account').resample('1D').mean().interpolate(method='linear').reset_index()
df_resampled = df_resampled.merge(accounts, on='account')
df_resampled


,account,date,balance,bank,type
0,account1,2023-04-14,5609.310000,BNP Paribas,checking
1,account1,2023-04-15,5609.046000,BNP Paribas,checking
2,account1,2023-04-16,5608.782000,BNP Paribas,checking
3,account1,2023-04-17,5608.518000,BNP Paribas,checking
4,account1,2023-04-18,5608.254000,BNP Paribas,checking
...,...,...,...,...,...
3145,stocks2,2024-09-15,7339.283188,Degiro,stocks
3146,stocks2,2024-09-16,7359.607391,Degiro,stocks
3147,stocks2,2024-09-17,7379.931594,Degiro,stocks
3148,stocks2,2024-09-18,7400.255797,Degiro,stocks


In [6]:
total = df_resampled.groupby(['date'])['balance'].sum().reset_index()

fig = px.area(total, x='date', y='balance', title='Total Portfolio Value Over Time')
fig

In [7]:
category_order = ['checking', 'savings', 'stocks', 'crypto']
fig = px.area(df_resampled, x='date', y='balance', color='type', line_group='account', category_orders={'type': category_order}, title='Total Portfolio Value Over Time')
fig